# Calcoli Strutturali - Stampante 3D Custom 900×740×800mm

Notebook interattivo per dimensionamento e verifica strutturale della stampante 3D.

**Specifiche progetto:**
- **Area di stampa**: 900×740×800mm (X×Y×Z)
- **Dimensioni struttura**: 1080×1000×1100mm (X×Y×Z)
- **Profili acciaio**: 40×20×2mm
- **Materiale**: Acciaio S235 (E = 210 GPa)

**Cinematica:**
- **Asse Z**: Piatto riscaldato 30 kg mobile verticalmente (4 × NEMA23)
  - Vetro temperato 900×900×10mm (20 kg)
  - Piatto alluminio 900×900×4mm + 9 riscaldatori (10 kg)
- **Asse X**: Testina + estrusore 1 kg mobile orizzontalmente (1080mm)
- **Asse Y**: Binari NGM20H fissi (1000mm) su cui scorre l'asse X

In [ ]:
# Import librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Configurazione grafici
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Parametri Stampante

Definizione masse e dimensioni reali

In [ ]:
# ============================================================================
# DIMENSIONI STRUTTURA
# ============================================================================
# Area di stampa utile [mm]
area_stampa_X = 900
area_stampa_Y = 740
area_stampa_Z = 800

# Dimensioni struttura effettiva [mm]
struttura_X = 1080
struttura_Y = 1000
struttura_Z = 1100

print("📐 DIMENSIONI")
print(f"   Area di stampa: {area_stampa_X}×{area_stampa_Y}×{area_stampa_Z} mm")
print(f"   Struttura:      {struttura_X}×{struttura_Y}×{struttura_Z} mm")
print(f"   Margini:        X={struttura_X-area_stampa_X}mm, Y={struttura_Y-area_stampa_Y}mm, Z={struttura_Z-area_stampa_Z}mm")

# ============================================================================
# MASSE COMPONENTI [kg]
# ============================================================================
# Piatto riscaldato (mobile su asse Z)
m_vetro = 20.0         # Vetro temperato 900×900×10mm
m_alluminio = 7.0      # Piatto alluminio 900×900×4mm
m_riscaldatori = 3.0   # 9 riscaldatori + cablaggio
m_piatto_totale = m_vetro + m_alluminio + m_riscaldatori

# Testina mobile (mobile su asse X)
m_testina = 1.0        # Estrusore + hotend + supporto

# Guide e componenti fissi
m_binari_Y = 8.0       # Binari NGM20H asse Y (fissi)
m_guide_X = 4.0        # Guide lineari asse X
m_guide_Z = 12.0       # Guide lineari asse Z (4 montanti)

print("\n⚖️ MASSE")
print(f"   Piatto riscaldato (asse Z):")
print(f"   - Vetro: {m_vetro:.1f} kg")
print(f"   - Alluminio + riscaldatori: {m_alluminio + m_riscaldatori:.1f} kg")
print(f"   - Totale: {m_piatto_totale:.1f} kg")
print(f"\n   Testina (asse X): {m_testina:.1f} kg")
print(f"   Binari Y (fissi): {m_binari_Y:.1f} kg")

# ============================================================================
# ACCELERAZIONI [mm/s²]
# ============================================================================
a_max_X = 5000         # Testina leggera: accelerazioni elevate
a_max_Z = 500          # Piatto pesante: accelerazioni ridotte

print(f"\n🚀 ACCELERAZIONI")
print(f"   Asse X (testina): {a_max_X} mm/s² = {a_max_X/1000:.1f} m/s²")
print(f"   Asse Z (piatto):  {a_max_Z} mm/s² = {a_max_Z/1000:.1f} m/s²")

## 2. Calcolo Momento di Inerzia Profilo

Profilo rettangolare cavo 40×20×2mm

In [ ]:
# Dimensioni profilo [mm]
b_ext = 40  # Base esterna
h_ext = 20  # Altezza esterna
t = 2       # Spessore parete

b_int = b_ext - 2*t  # Base interna
h_int = h_ext - 2*t  # Altezza interna

# Momento di inerzia rispetto asse forte (flessione con base 40mm verticale)
I_forte = (b_ext * h_ext**3 - b_int * h_int**3) / 12

# Momento di inerzia rispetto asse debole (flessione con altezza 20mm verticale)
I_debole = (h_ext * b_ext**3 - h_int * b_int**3) / 12

# Area sezione
A = b_ext * h_ext - b_int * h_int

# Densità acciaio
rho = 7850e-9  # [kg/mm³]

print(f"📐 PROFILO: {b_ext}×{h_ext}×{t} mm")
print(f"   Area sezione: {A:.1f} mm²")
print(f"   Momento inerzia asse forte (40mm verticale): {I_forte:.1f} mm⁴")
print(f"   Momento inerzia asse debole (20mm verticale): {I_debole:.1f} mm⁴")
print(f"   Rapporto I_forte/I_debole: {I_forte/I_debole:.2f}x")
print(f"\n   Massa lineare: {A * rho * 1000:.2f} g/mm")

## 3. Analisi Asse X - Trave 1080mm (Testina Mobile)

Trave che supporta la testina da 1 kg in movimento orizzontale.

**Modello**: Trave appoggiata agli estremi con carico concentrato al centro

In [ ]:
# Parametri trave asse X
L_X = struttura_X  # 1080 mm
E = 210000         # Modulo elasticità acciaio [N/mm²]
I_X = I_debole     # Caso critico: momento inerzia debole

# Carichi
P_testina = m_testina * 9.81                    # Peso statico [N]
F_inerzia_X = (m_testina * a_max_X) / 1000      # Forza inerziale [N]
F_totale_X = P_testina + F_inerzia_X            # Carico totale equivalente [N]

# Flessione (formula trave appoggiata, carico centrato)
# δ = F·L³ / (48·E·I)
flessione_X = (F_totale_X * L_X**3) / (48 * E * I_X)

# Limite tolleranza (rigidità stampanti 3D: L/500)
flessione_limite_X = L_X / 500

print("="*70)
print("  🔧 ANALISI ASSE X - TRAVE 1080mm (Supporto Testina)")
print("="*70)
print(f"\n📏 Geometria:")
print(f"   Lunghezza trave: {L_X} mm")
print(f"   Profilo: {b_ext}×{h_ext}×{t} mm")
print(f"   Momento inerzia: {I_X:.1f} mm⁴ (asse debole - caso critico)")

print(f"\n⚖️ Carichi:")
print(f"   Massa testina: {m_testina:.1f} kg")
print(f"   Peso statico: {P_testina:.2f} N")
print(f"   Accelerazione max: {a_max_X} mm/s² = {a_max_X/1000:.1f} m/s²")
print(f"   Forza inerziale: {F_inerzia_X:.2f} N")
print(f"   Carico totale equivalente: {F_totale_X:.2f} N")

print(f"\n📐 Flessione:")
print(f"   Calcolata: {flessione_X:.3f} mm")
print(f"   Limite ammissibile (L/500): {flessione_limite_X:.3f} mm")
print(f"   Utilizzo: {(flessione_X/flessione_limite_X)*100:.1f}%")

if flessione_X < flessione_limite_X:
    print(f"   ✅ OK: Flessione entro limiti (margine {flessione_limite_X - flessione_X:.3f} mm)")
else:
    print(f"   ⚠️ ATTENZIONE: Flessione eccessiva (superamento {flessione_X - flessione_limite_X:.3f} mm)")
    
# Carico massimo ammissibile
F_max_X = (flessione_limite_X * 48 * E * I_X) / L_X**3
print(f"\n💪 Carico massimo ammissibile: {F_max_X:.1f} N")
print(f"   Coefficiente sicurezza: {F_max_X / F_totale_X:.2f}x")

## 4. Analisi Asse Y - Trave 1000mm (Supporto Binari)

Trave che supporta i binari fissi NGM20H su cui scorre l'asse X.

**Carico**: Binari + testina in posizione critica centrale

In [ ]:
# Parametri trave asse Y
L_Y = struttura_Y  # 1000 mm
I_Y = I_debole     # Caso critico

# Carichi (binari fissi + testina quando al centro)
P_binari = m_binari_Y * 9.81                    # Peso binari [N]
P_testina_su_Y = m_testina * 9.81               # Peso testina [N]
F_inerzia_Y = (m_testina * a_max_X) / 1000      # Forza inerzia testina [N]
F_totale_Y = P_binari + P_testina_su_Y + F_inerzia_Y  # Totale [N]

# Flessione
flessione_Y = (F_totale_Y * L_Y**3) / (48 * E * I_Y)
flessione_limite_Y = L_Y / 500

print("="*70)
print("  🔧 ANALISI ASSE Y - TRAVE 1000mm (Supporto Binari)")
print("="*70)
print(f"\n📏 Geometria:")
print(f"   Lunghezza trave: {L_Y} mm")
print(f"   Momento inerzia: {I_Y:.1f} mm⁴")

print(f"\n⚖️ Carichi:")
print(f"   Binari NGM20H (fissi): {m_binari_Y:.1f} kg → {P_binari:.2f} N")
print(f"   Testina (al centro): {m_testina:.1f} kg → {P_testina_su_Y:.2f} N")
print(f"   Forza inerziale testina: {F_inerzia_Y:.2f} N")
print(f"   Carico totale: {F_totale_Y:.2f} N")

print(f"\n📐 Flessione:")
print(f"   Calcolata: {flessione_Y:.3f} mm")
print(f"   Limite ammissibile (L/500): {flessione_limite_Y:.3f} mm")
print(f"   Utilizzo: {(flessione_Y/flessione_limite_Y)*100:.1f}%")

if flessione_Y < flessione_limite_Y:
    print(f"   ✅ OK: Flessione entro limiti (margine {flessione_limite_Y - flessione_Y:.3f} mm)")
else:
    print(f"   ⚠️ ATTENZIONE: Flessione eccessiva")

F_max_Y = (flessione_limite_Y * 48 * E * I_Y) / L_Y**3
print(f"\n💪 Carico massimo ammissibile: {F_max_Y:.1f} N")
print(f"   Coefficiente sicurezza: {F_max_Y / F_totale_Y:.2f}x")

## 5. Analisi Asse Z - Montanti Verticali 1100mm (Piatto 30kg)

**Tipo di carico**: Compressione assiale + momento flettente da accelerazione orizzontale

I montanti verticali supportano:
- Peso statico piatto: 30 kg
- Forza inerziale verticale durante accelerazione Z
- Momento flettente da vibrazioni/accelerazioni orizzontali

In [ ]:
# Parametri montanti verticali
L_Z = struttura_Z  # 1100 mm
I_Z = I_forte      # Uso asse forte per montanti verticali (più rigidi)
n_montanti = 4     # 4 montanti agli angoli

# Carichi verticali (compressione)
P_piatto = m_piatto_totale * 9.81                      # Peso piatto [N]
F_inerzia_Z = (m_piatto_totale * a_max_Z) / 1000       # Forza inerziale [N]
F_compressione_totale = P_piatto + F_inerzia_Z         # Totale [N]
F_per_montante = F_compressione_totale / n_montanti    # Per montante [N]

# Carico critico di Eulero (carico di punta)
# P_cr = π²·E·I / L²  (trave vincolata agli estremi)
P_critico = (np.pi**2 * E * I_Z) / L_Z**2
coeff_sicurezza_punta = P_critico / F_per_montante

# Flessione laterale da vibrazioni (stima conservativa: 1% del peso come forza orizzontale)
F_orizzontale = P_piatto * 0.01  # 1% del peso
flessione_Z_laterale = (F_orizzontale * L_Z**3) / (48 * E * I_Z)

print("="*70)
print("  🔧 ANALISI ASSE Z - MONTANTI VERTICALI 1100mm (Piatto 30kg)")
print("="*70)
print(f"\n📏 Geometria:")
print(f"   Altezza montanti: {L_Z} mm")
print(f"   Numero montanti: {n_montanti}")
print(f"   Momento inerzia (asse forte): {I_Z:.1f} mm⁴")

print(f"\n⚖️ Carichi verticali (compressione):")
print(f"   Massa piatto: {m_piatto_totale:.1f} kg")
print(f"   Peso statico: {P_piatto:.2f} N")
print(f"   Accelerazione max Z: {a_max_Z} mm/s²")
print(f"   Forza inerziale: {F_inerzia_Z:.2f} N")
print(f"   Carico totale: {F_compressione_totale:.2f} N")
print(f"   Carico per montante: {F_per_montante:.2f} N")

print(f"\n🏗️ Verifica stabilità (carico di punta):")
print(f"   Carico critico Eulero: {P_critico:.1f} N/montante")
print(f"   Coefficiente sicurezza: {coeff_sicurezza_punta:.1f}x")

if coeff_sicurezza_punta > 3:
    print(f"   ✅ OK: Ampio margine di sicurezza contro instabilità")
elif coeff_sicurezza_punta > 2:
    print(f"   ✅ OK: Margine di sicurezza adeguato")
else:
    print(f"   ⚠️ ATTENZIONE: Rischio instabilità, aumentare sezione")

print(f"\n📐 Flessione laterale (vibrazioni 1% peso):")
print(f"   Forza orizzontale stimata: {F_orizzontale:.2f} N")
print(f"   Flessione laterale: {flessione_Z_laterale:.3f} mm")
print(f"   {'✅ Trascurabile' if flessione_Z_laterale < 0.1 else '⚠️ Da considerare'}")

## 6. Analisi Frequenze di Risonanza

Verifica che le frequenze naturali della struttura non coincidano con le frequenze operative dei motori.

In [ ]:
# Massa lineare trave
mu = A * rho  # [kg/mm]

# Coefficiente primo modo (trave appoggiata)
lambda_1 = 3.927

# Frequenze naturali [Hz]
f_naturale_X = (lambda_1**2 / (2 * np.pi * L_X**2)) * np.sqrt((E * I_X) / mu)
f_naturale_Y = (lambda_1**2 / (2 * np.pi * L_Y**2)) * np.sqrt((E * I_Y) / mu)
f_naturale_Z = (lambda_1**2 / (2 * np.pi * L_Z**2)) * np.sqrt((E * I_Z) / mu)

# Frequenze operative motori
# Esempio: NEMA23 a 200 step/rev, 16 microstepping, velocità max 150 mm/s
steps_per_rev = 200 * 16  # 3200 step/rev con microstepping
pitch_leadscrew = 5       # Lead screw TR8x5: 5mm/rev
v_max = 150               # Velocità max [mm/s]
rpm_max = (v_max / pitch_leadscrew) * 60  # RPM massimi
freq_motore = (rpm_max / 60) * steps_per_rev  # Frequenza step [Hz]

print("="*70)
print("  🎵 ANALISI FREQUENZE DI RISONANZA")
print("="*70)

print(f"\n📐 Frequenze naturali struttura:")
print(f"   Asse X ({L_X}mm): {f_naturale_X:.2f} Hz")
print(f"   Asse Y ({L_Y}mm): {f_naturale_Y:.2f} Hz")
print(f"   Asse Z ({L_Z}mm): {f_naturale_Z:.2f} Hz")

print(f"\n⚙️ Frequenze operative motori:")
print(f"   Step/rev: {steps_per_rev}")
print(f"   Velocità max: {v_max} mm/s")
print(f"   RPM max: {rpm_max:.1f}")
print(f"   Frequenza step: {freq_motore:.0f} Hz")

print(f"\n✅ Verifica risonanza:")
margine_X = abs(f_naturale_X - freq_motore) / freq_motore * 100
margine_Y = abs(f_naturale_Y - freq_motore) / freq_motore * 100
margine_Z = abs(f_naturale_Z - freq_motore) / freq_motore * 100

print(f"   Asse X: {margine_X:.1f}% separazione → {'✅ OK' if margine_X > 20 else '⚠️ ATTENZIONE'}")
print(f"   Asse Y: {margine_Y:.1f}% separazione → {'✅ OK' if margine_Y > 20 else '⚠️ ATTENZIONE'}")
print(f"   Asse Z: {margine_Z:.1f}% separazione → {'✅ OK' if margine_Z > 20 else '⚠️ ATTENZIONE'}")

## 7. Confronto Profili Alternativi

Analisi comparativa per valutare profili alternativi al 40×20×2mm

In [ ]:
# Database profili [b, h, t] mm
profili = {
    '30×20×2': (30, 20, 2),
    '40×20×2': (40, 20, 2),  # Profilo attuale
    '40×30×2': (40, 30, 2),
    '40×40×2': (40, 40, 2),
    '50×30×2': (50, 30, 2),
    '60×40×3': (60, 40, 3),
}

risultati = []

for nome, (b, h, s) in profili.items():
    # Calcolo proprietà
    b_i = b - 2*s
    h_i = h - 2*s
    I_calc = (h * b**3 - h_i * b_i**3) / 12  # Momento inerzia debole
    A_calc = b * h - b_i * h_i
    
    # Flessione asse X (caso critico: trave 1080mm, carico testina)
    flex_X = (F_totale_X * L_X**3) / (48 * E * I_calc)
    
    # Flessione asse Y
    flex_Y = (F_totale_Y * L_Y**3) / (48 * E * I_calc)
    
    # Massa
    massa_X = A_calc * L_X * rho * 1000  # [g]
    
    # Carico critico montante Z
    I_forte_calc = (b * h**3 - b_i * h_i**3) / 12
    P_crit = (np.pi**2 * E * I_forte_calc) / L_Z**2
    
    risultati.append({
        'Profilo': nome,
        'Area [mm²]': A_calc,
        'I_debole [mm⁴]': I_calc,
        'Fless X [mm]': flex_X,
        'Fless Y [mm]': flex_Y,
        'Massa X [g]': massa_X,
        'P_crit Z [N]': P_crit,
        'OK_X': '✅' if flex_X < flessione_limite_X else '❌',
        'OK_Y': '✅' if flex_Y < flessione_limite_Y else '❌'
    })

# Visualizzazione risultati
import pandas as pd
df = pd.DataFrame(risultati)
df = df.set_index('Profilo')

print("\n" + "="*100)
print("  📋 CONFRONTO PROFILI ALTERNATIVI")
print("="*100 + "\n")
print(df.to_string())

# Grafici confronto
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 10))

# 1. Flessione Asse X
colors_X = ['green' if x == '✅' else 'red' for x in df['OK_X']]
ax1.bar(df.index, df['Fless X [mm]'], color=colors_X, alpha=0.7)
ax1.axhline(y=flessione_limite_X, color='orange', linestyle='--', linewidth=2, label=f'Limite = {flessione_limite_X:.2f} mm')
ax1.set_ylabel('Flessione [mm]', fontsize=11)
ax1.set_title(f'Flessione Asse X (Trave {L_X}mm, Testina {m_testina}kg)', fontsize=12, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Flessione Asse Y
colors_Y = ['green' if x == '✅' else 'red' for x in df['OK_Y']]
ax2.bar(df.index, df['Fless Y [mm]'], color=colors_Y, alpha=0.7)
ax2.axhline(y=flessione_limite_Y, color='orange', linestyle='--', linewidth=2, label=f'Limite = {flessione_limite_Y:.2f} mm')
ax2.set_ylabel('Flessione [mm]', fontsize=11)
ax2.set_title(f'Flessione Asse Y (Trave {L_Y}mm, Binari+Testina)', fontsize=12, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Massa trave X
colors_massa = ['blue' if '40×20×2' in x else 'gray' for x in df.index]
ax3.bar(df.index, df['Massa X [g]'], color=colors_massa, alpha=0.7)
ax3.set_ylabel('Massa [g]', fontsize=11)
ax3.set_title(f'Massa Trave Asse X (L={L_X}mm)', fontsize=12, fontweight='bold')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

# 4. Carico critico montante Z
ax4.bar(df.index, df['P_crit Z [N]'], color='darkgreen', alpha=0.7)
ax4.axhline(y=F_per_montante, color='red', linestyle='--', linewidth=2, label=f'Carico reale = {F_per_montante:.1f} N')
ax4.set_ylabel('Carico Critico [N]', fontsize=11)
ax4.set_title(f'Carico Critico Eulero - Montante Z (L={L_Z}mm)', fontsize=12, fontweight='bold')
ax4.tick_params(axis='x', rotation=45)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Raccomandazioni
print("\n💡 RACCOMANDAZIONI:")
profilo_ottimale = df[(df['OK_X'] == '✅') & (df['OK_Y'] == '✅')].sort_values('Massa X [g]').index[0]
print(f"   Profilo più leggero OK: {profilo_ottimale}")
print(f"   Profilo attuale (40×20×2): {'✅ Adeguato' if df.loc['40×20×2', 'OK_X'] == '✅' and df.loc['40×20×2', 'OK_Y'] == '✅' else '❌ Insufficiente'}")

## 8. Grafico Parametrico: Flessione vs Accelerazione

In [ ]:
# Range accelerazioni da testare [mm/s²]
accelerazioni = np.linspace(1000, 10000, 100)

# Calcolo flessioni per diverse accelerazioni
flessioni_X_acc = []
for a in accelerazioni:
    F_tot = m_testina * 9.81 + (m_testina * a) / 1000
    flex = (F_tot * L_X**3) / (48 * E * I_X)
    flessioni_X_acc.append(flex)

# Grafico
plt.figure(figsize=(12, 6))
plt.plot(accelerazioni, flessioni_X_acc, 'b-', linewidth=2, label=f'Flessione asse X (testina {m_testina}kg)')
plt.axhline(y=flessione_limite_X, color='r', linestyle='--', linewidth=2, label=f'Limite L/500 = {flessione_limite_X:.2f} mm')
plt.axvline(x=a_max_X, color='g', linestyle=':', linewidth=2, label=f'Accelerazione progetto = {a_max_X} mm/s²')

plt.xlabel('Accelerazione [mm/s²]', fontsize=12)
plt.ylabel('Flessione [mm]', fontsize=12)
plt.title(f'Flessione Asse X vs Accelerazione (Trave {L_X}mm, Profilo {b_ext}×{h_ext}×{t})', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

# Accelerazione massima ammissibile
F_peso = m_testina * 9.81
a_max_ammissibile = ((flessione_limite_X * 48 * E * I_X / L_X**3) - F_peso) * 1000 / m_testina
print(f"\n⚡ Accelerazione massima ammissibile (flessione < L/500): {a_max_ammissibile:.0f} mm/s²")
print(f"   Margine rispetto progetto ({a_max_X} mm/s²): {(a_max_ammissibile/a_max_X - 1)*100:.1f}%")

## 9. Riepilogo Finale e Conclusioni

In [ ]:
print("\n" + "="*80)
print("  📊 RIEPILOGO COMPLETO - ANALISI STRUTTURALE STAMPANTE 3D")
print("="*80)

print(f"\n🏗️ CONFIGURAZIONE:")
print(f"   Area stampa: {area_stampa_X}×{area_stampa_Y}×{area_stampa_Z} mm")
print(f"   Struttura:   {struttura_X}×{struttura_Y}×{struttura_Z} mm")
print(f"   Profilo:     {b_ext}×{h_ext}×{t} mm (Acciaio S235)")

print(f"\n⚖️ MASSE:")
print(f"   Piatto riscaldato (Z): {m_piatto_totale:.1f} kg")
print(f"   Testina mobile (X):    {m_testina:.1f} kg")
print(f"   Binari Y (fissi):      {m_binari_Y:.1f} kg")

print(f"\n🚀 ACCELERAZIONI:")
print(f"   Asse X: {a_max_X} mm/s² ({a_max_X/1000:.1f} m/s²)")
print(f"   Asse Z: {a_max_Z} mm/s² ({a_max_Z/1000:.1f} m/s²)")

print(f"\n" + "-"*80)
print(f"  ASSE X - Trave {L_X}mm (Testina Mobile)")
print("-"*80)
print(f"   Carico totale:    {F_totale_X:.2f} N")
print(f"   Flessione:        {flessione_X:.3f} mm (limite {flessione_limite_X:.3f} mm)")
print(f"   Status:           {'✅ OK - Entro limiti' if flessione_X < flessione_limite_X else '⚠️ CRITICO'}")
print(f"   Utilizzo:         {(flessione_X/flessione_limite_X)*100:.1f}%")
print(f"   Freq. naturale:   {f_naturale_X:.2f} Hz")

print(f"\n" + "-"*80)
print(f"  ASSE Y - Trave {L_Y}mm (Binari Fissi)")
print("-"*80)
print(f"   Carico totale:    {F_totale_Y:.2f} N")
print(f"   Flessione:        {flessione_Y:.3f} mm (limite {flessione_limite_Y:.3f} mm)")
print(f"   Status:           {'✅ OK - Entro limiti' if flessione_Y < flessione_limite_Y else '⚠️ CRITICO'}")
print(f"   Utilizzo:         {(flessione_Y/flessione_limite_Y)*100:.1f}%")
print(f"   Freq. naturale:   {f_naturale_Y:.2f} Hz")

print(f"\n" + "-"*80)
print(f"  ASSE Z - Montanti {L_Z}mm (Piatto {m_piatto_totale}kg)")
print("-"*80)
print(f"   Carico/montante:  {F_per_montante:.2f} N ({n_montanti} montanti)")
print(f"   Carico critico:   {P_critico:.1f} N/montante")
print(f"   Coeff. sicur.:    {coeff_sicurezza_punta:.1f}x")
print(f"   Status:           {'✅ OK - Stabile' if coeff_sicurezza_punta > 2 else '⚠️ Verificare'}")
print(f"   Freq. naturale:   {f_naturale_Z:.2f} Hz")

print(f"\n🎵 RISONANZA:")
print(f"   Freq. operativa motori: {freq_motore:.0f} Hz")
print(f"   Separazione freq. X:    {margine_X:.1f}% → {'✅ OK' if margine_X > 20 else '⚠️ ATTENZIONE'}")
print(f"   Separazione freq. Y:    {margine_Y:.1f}% → {'✅ OK' if margine_Y > 20 else '⚠️ ATTENZIONE'}")
print(f"   Separazione freq. Z:    {margine_Z:.1f}% → {'✅ OK' if margine_Z > 20 else '⚠️ ATTENZIONE'}")

print(f"\n💰 STIMA MASSA STRUTTURA:")
travi_totali = 12  # Stima conservativa
massa_struttura = (A * ((L_X * 2) + (L_Y * 2) + (L_Z * n_montanti)) * rho * 1000) / 1000
print(f"   Massa profili principali: {massa_struttura:.1f} kg")

# Valutazione finale
print(f"\n" + "="*80)
ok_X = flessione_X < flessione_limite_X
ok_Y = flessione_Y < flessione_limite_Y
ok_Z = coeff_sicurezza_punta > 2

if ok_X and ok_Y and ok_Z:
    print("  ✅ STRUTTURA ADEGUATA - Tutti i criteri soddisfatti")
elif ok_X and ok_Y:
    print("  ⚠️ STRUTTURA ACCETTABILE - Verificare montanti Z")
else:
    print("  ❌ STRUTTURA CRITICA - Considerare profili più robusti")
print("="*80 + "\n")

---

## 📝 Note Tecniche

### Formule Utilizzate

**Flessione trave appoggiata (carico concentrato al centro):**
$$\delta = \frac{F \cdot L^3}{48 \cdot E \cdot I}$$

**Momento di inerzia profilo rettangolare cavo:**
$$I = \frac{b \cdot h^3 - b' \cdot h'^3}{12}$$

**Carico critico di Eulero (instabilità):**
$$P_{cr} = \frac{\pi^2 \cdot E \cdot I}{L^2}$$

**Frequenza naturale trave:**
$$f = \frac{\lambda^2}{2\pi L^2} \sqrt{\frac{EI}{\mu}}$$
dove λ = 3.927 (primo modo, trave appoggiata)

### Ipotesi e Limiti

**Ipotesi:**
- Travi appoggiate agli estremi (vincoli semplici)
- Materiale elastico lineare (acciaio S235, E = 210 GPa)
- Carico statico equivalente (peso + forza inerziale)
- Deformazioni piccole (teoria trave di Eulero-Bernoulli)
- Carico concentrato al centro (caso peggiore)

**Limiti di validità:**
- Flessione < L/500 (rigidità richiesta per stampanti 3D)
- Stress < σ_snervamento (non calcolato esplicitamente, design conservativo)
- Frequenze operative < f_naturale/2 (margine sicurezza risonanza)
- Coefficiente sicurezza punta > 2 (instabilità montanti)

### Cinematica Stampante

```
┌─────────────────────────────────────┐
│  CONFIGURAZIONE ASSI                │
│                                     │
│  ┌──────────────┐                   │
│  │  Testina 1kg │ ◄──── Asse X      │
│  └──────┬───────┘       (1080mm)    │
│         │                           │
│    Binari Y (fissi, 1000mm)         │
│         │                           │
│  ┌──────▼───────┐                   │
│  │ Piatto 30kg  │ ◄──── Asse Z      │
│  │  (mobile)    │       (1100mm)    │
│  └──────────────┘                   │
│    4 × NEMA23                       │
└─────────────────────────────────────┘
```

---

*Notebook aggiornato: 2026-01-25*  
*Configurazione: Stampante 3D Custom - Area stampa 900×740×800mm - Struttura 1080×1000×1100mm*  
*Cinematica: Piatto mobile Z (30kg), Testina mobile X (1kg), Binari Y fissi*